<a href="https://colab.research.google.com/github/WoobeenJeong/bioinfo1_jwb/blob/main/Project_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Data mounting**

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2) Environment setting**

In [ ]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

In [ ]:
!conda install -y bedtools bioawk samtools

**3) Data loading & check**

In [10]:
%cd /content/drive/MyDrive/binfo1-work

/content/drive/MyDrive/binfo1-work


In [ ]:
!ls -al /content/drive/MyDrive/binfo1-work/

In [ ]:
!md5sum /content/drive/MyDrive/binfo1-work/*

**4) Background setting**

In [3]:
%cd /content/drive/MyDrive/binfo1-work
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import Counter

/content/drive/MyDrive/binfo1-work


**5) Target region on CLIP-seq using gencode.gtf**

먼저 GTF파일은 genecode로부터 M32버전이 (2023.02월에) 나왔으며  
M32(GRCm39) Refseq 자료 중 chr9에 대해 확인 할 것  
NCBI RefSeq에서도 Mus musculus strain C57BL/6J chromosome 9, GRCm39 존재

1) C=Shannon’s entrophy를 CRES: Crosslinking-induced reverse transcription error score로 가져감  
2) 이때, insertion은 무시 substitution과 deletion고려  
3) FDR은 permutation으로 구함   
4) Read는 rRNA,tRNA,adapte제외하고 GSNAP의 Refseq에 align  
5) 이를 바탕으로, 먼저 CRES(0.8) 상 low depth positio이 130 reads 이하  
6) LOESS smoothing (regression) a=0.5사용 medium depth positon 250 read이하  
7) FDR(0.001)상 high depth position >250 read 사용  

In [11]:
!gzip -d gencode.vM32.annotation.gtf.gz

In [15]:
!grep 'chr9' gencode.vM32.annotation.gtf > gencode.vM32.chr9.gtf

In [16]:
!head gencode.vM32.chr9.gtf

chr9	ENSEMBL	gene	3038669	3038743	.	-	.	gene_id "ENSMUSG00000106183.2"; gene_type "miRNA"; gene_name "Mir101c"; level 3; mgi_id "MGI:4950030";
chr9	ENSEMBL	transcript	3038669	3038743	.	-	.	gene_id "ENSMUSG00000106183.2"; transcript_id "ENSMUST00000195989.2"; gene_type "miRNA"; gene_name "Mir101c"; transcript_type "miRNA"; transcript_name "Mir101c-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:4950030"; tag "basic"; tag "Ensembl_canonical";
chr9	ENSEMBL	exon	3038669	3038743	.	-	.	gene_id "ENSMUSG00000106183.2"; transcript_id "ENSMUST00000195989.2"; gene_type "miRNA"; gene_name "Mir101c"; transcript_type "miRNA"; transcript_name "Mir101c-201"; exon_number 1; exon_id "ENSMUSE00001348199.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:4950030"; tag "basic"; tag "Ensembl_canonical";
chr9	HAVANA	gene	3133847	3199799	.	-	.	gene_id "ENSMUSG00000042360.13"; gene_type "lncRNA"; gene_name "4930433N12Rik"; level 2; mgi_id "MGI:2149746"; tag "overlapping_locus"; havana_gene "OTTM

In [33]:
!awk -F'\t' '{if ($3 == "gene") print $4}' gencode.vM32.chr9.gtf | sort -n | head -n 1
!awk -F'\t' '{if ($3 == "gene") print $4}' gencode.vM32.chr9.gtf | sort -n | tail -n 1

3038669
124252714


In [13]:
!samtools view -b -o CLIP-let7g.bam CLIP-35L33G.bam chr9:3038669-124252714
!samtools view CLIP-let7g.bam | wc -l

[E::hts_open_format] Failed to open file CLIP-35L33G.bam
samtools view: failed to open "CLIP-35L33G.bam" for reading: No such file or directory
1696159


In [14]:
!samtools mpileup CLIP-let7g.bam 2>/dev/null > CLIP-let7g.pileup
!wc -l CLIP-let7g.pileup

47303563 CLIP-let7g.pileup


In [15]:
!awk '$2 >= 3038669 && $2 <= 124252714 { print $0; }' CLIP-let7g.pileup > CLIP-let7g-gene.pileup
!tail -n 2 CLIP-let7g-gene.pileup

chr9	124252286	N	1	c	E
chr9	124252287	N	1	c$	D


In [16]:
pileup = pd.read_csv('CLIP-let7g-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

,chrom,pos,_ref,count,basereads,quals
47303558,chr9,124252283,N,1,g,H
47303559,chr9,124252284,N,1,g,G
47303560,chr9,124252285,N,1,a,F
47303561,chr9,124252286,N,1,c,E
47303562,chr9,124252287,N,1,c$,D


In [ ]:
count의 개수가 최대 / 최소 얼마이고 볼 만한 수준인지 확인

In [17]:
len(np.unique(pileup['count']))

2443

In [44]:
pileup['count'].describe()

count    4.730356e+07
mean     1.007026e+01
std      3.933483e+01
min      0.000000e+00
25%      3.000000e+00
50%      5.000000e+00
75%      1.000000e+01
max      7.983000e+03
Name: count, dtype: float64

In [47]:
np.max(pileup['count'])

7983

위를 보면 알수 있듯이 한 75%까지는 10개정도만 count되어 있는데, 무려 7983 read가 count되는 경우도 있음.  
이경우는 좀 궁금하긴 하다만, 너무 크기 때문에, threshold <=130 reads로 두고 진행   
(Shannon에서 depth 130으로 했으므로)

In [18]:
filter130pileup = pileup[pileup['count'] <= 130]

In [19]:
len(np.unique(filter130pileup['count']))

131

In [20]:
len(np.unique(filter130pileup['pos']))

47076399

In [21]:
filter130pileup['count'].describe()

count    4.707640e+07
mean     8.371837e+00
std      1.172930e+01
min      0.000000e+00
25%      3.000000e+00
50%      5.000000e+00
75%      9.000000e+00
max      1.300000e+02
Name: count, dtype: float64

In [23]:
max(filter130pileup['pos']),max(filter130pileup['pos'])-min(filter130pileup['pos'])

(124252287, 121161794)

match와 substitution, deletion을 사용할 것이므로, 나머지에 해당하는 tag들은 제거.  
**여기서 Deletion은 포함하는 경우를 고려하는 것을 고민할 것**

In [24]:
toremove = re.compile('[<>$*#^]')
filter130pileup.loc[:, 'matches'] = filter130pileup['basereads'].apply(lambda x: toremove.sub('', x))

<ipython-input-24-a83007237d9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter130pileup.loc[:, 'matches'] = filter130pileup['basereads'].apply(lambda x: toremove.sub('', x))


In [26]:
filter130pileup[['chrom', 'pos', 'matches']].head()

,chrom,pos,matches
0,chr9,3090493,HAIAGAHA
1,chr9,3090494,GGGG
2,chr9,3090495,GGGG
3,chr9,3090496,GGGG
4,chr9,3090497,GGGG


In [ ]:
x=filter130pileup['pos']
y=filter130pileup['count']
plt.bar(x,y)
plt.xlabel('Position')
plt.ylabel('Count')
plt.title('Distribution of reads counts on position(chr=9)')
plt.show()

다행히 진짜 mirleg7 포지션에 매칭은 됨

In [29]:
filter130pileup[filter130pileup['pos'] == 106056094].iloc[0]['matches']

'GGGGGGAAAAAAAAGGGGGAAAAAAGCCGCAGGATGAGGTGATAAGGGAGGGGTGAAGGGCGGTGAAGGGGAAAAGAGAAAGAAAAATAAAGGGGGAGTGGGAGGAAGAAGAGAATA'

Deletion상관 없이 Insertion하고 Hard-clipped read만 제외

In [30]:
filter130pileup['base_count'] = filter130pileup['matches'].apply(lambda x: len([c for c in x if c != 'H' and c != 'I']))
filter130pileup.head()

<ipython-input-30-80f731dc1106>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter130pileup['base_count'] = filter130pileup['matches'].apply(lambda x: len([c for c in x if c != 'H' and c != 'I']))


,chrom,pos,_ref,count,basereads,quals,matches,base_count
0,chr9,3090493,N,4,^HA^IA^GA^HA,HIEG,HAIAGAHA,5
1,chr9,3090494,N,4,GGGG,HIGG,GGGG,4
2,chr9,3090495,N,4,GGGG,HIGG,GGGG,4
3,chr9,3090496,N,4,GGGG,GHEG,GGGG,4
4,chr9,3090497,N,4,GGGG,HIGG,GGGG,4


In [31]:
np.unique(filter130pileup['base_count'])

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 161, 162, 163, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18

Colab RAM사용 제한 상, 데이터가 날라가는 것을 방지하기 위해 일단 결과 csv 저장

In [32]:
filter130pileup.to_csv('filter130pileup.csv', index=True)

RAM 사용량 상, 작업을 다른 주피터 노트북에 작성